<a href="https://colab.research.google.com/github/lucasART430/1-chatbot-Imersao-IA-Google-Alura/blob/main/sistema_para_busca_em_documentos_usando_embeddings_e_a_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install -q -U google-generativeai

In [19]:
#importações e configurações iniciais
import numpy as np # cálculos
import pandas as pd #excel ou google sheets
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

Embbedings: forma de representar palavras (tokens)
Guia de embbedings: https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br

In [32]:
for m in genai.list_models(): # listar os modelos disponiveis
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [33]:
#exemplo de embbeding
title = "A próxima geração de IA para desenvolvedores e Google"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google"
        "\n"
        "Artigo completo:\n"
        "\n"
        "Gemini API & Google: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                                            content=sample_text,
                                                            title=title,
                                                            task_type="RETRIEVAL_DOCUMENT")
print(embeddings)

{'embedding': [0.037907127, -0.029767027, -0.021151235, 0.040880665, 0.07469761, -0.004447404, -0.032306634, -0.018039009, 0.044220198, 0.054623667, 0.016382284, 0.0011507086, -0.06393731, -0.01628611, 0.008318366, -0.0056682155, 0.024255717, -0.021740008, -0.028648278, -0.019008774, -0.015676556, 0.018178921, -0.033113148, -0.05407111, -0.017341193, 0.019890932, 0.01724863, -0.04346004, -0.02928715, 0.01731339, -0.033916686, 0.042463385, -0.0331668, 0.0095756315, -0.03423262, -0.055345982, -0.058230106, -0.05565363, -0.0019214561, 0.016995274, -0.0003521671, -0.078646235, -4.5875247e-05, 0.034749802, -0.004020962, -0.022830633, 0.059096813, 0.04156028, 0.030719075, -0.06293682, 0.03150699, 0.029863933, 0.07375711, -0.044566713, -0.009614931, -0.008337214, 0.032474186, -0.031278994, 0.020445148, -0.007899833, 0.0032991702, 0.011999129, -0.0023137177, 0.036475055, 0.008009517, -0.03512294, -0.033759326, -0.0026217895, 0.028440867, 0.03565075, -0.0074685556, 0.01070621, 0.045042634, -0.0

Pesquisa de documentos com embeddings
Tipos de embbedings: https://ai.google.dev/gemini-api/tutorials/document_search?hl=pt-br#api_changes_to_embeddings_with_model_embedding-001

In [34]:
#listagem de documentos que serão buscados
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [35]:
df = pd.DataFrame(documents)
df.columns = ["Título", "Conteúdo"]
df

,Título,Conteúdo
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [36]:
model = "models/embedding-001"

In [37]:
def embed_fn(title, text):
  return genai.embed_content(model="models/embedding-001",
                                                            content=text,
                                                            title=title,
                                                            task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [38]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [39]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                                            content = consulta,
                                                            task_type="RETRIEVAL_QUERY")["embedding"]
  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [40]:
consulta = "Como faço para trocar marchas em um carro do Google?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.


In [42]:
generation_config = {
    "temperature": 0.5,
    "candidate_count": 1,
}

In [43]:
prompt = f"Reescreva esse texto de uma forma mais descontraida, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                                        generation_config=generation_config)

response = model_2.generate_content(prompt)
print(response.text)

Bora lá, no seu Googlecar tem uma telona irada que te dá acesso a um monte de coisa legal: mapa, músicas e até o ar-condicionado. É só tocar no ícone que você quer usar. Por exemplo, quer saber como chegar em algum lugar? Toca no "Mapa". Quer ouvir aquela musiquinha que você ama? Toca em "Música". Simples assim!
